In [7]:
import sys
sys.path.append("../")

In [8]:
import random 
import librosa
import numpy as np
import matplotlib.pyplot as plt
import libs.AudioDataSet

NameError: name '_C' is not defined

In [ ]:
def jaccard_similarity(signature1, signature2):
    """
    Compute the Jaccard similarity between two MinHash signatures.

    Parameters:
    signature1 (numpy array): MinHash signature of shape (num_hashes,)
    signature2 (numpy array): MinHash signature of shape (num_hashes,)

    Returns:
    similarity (float): Jaccard similarity between the two signatures
    """
    intersection = np.sum(np.bitwise_and(signature1, signature2))
    union = np.sum(np.bitwise_or(signature1, signature2))
    return intersection / union

def minHASH(arrays, num_perm):
  permutations = []
  count = 0
  while count < num_perm:
    perm = random.sample(range(0, len(arrays[0])), len(arrays[0]))
      # print(skm.mutual_info_score(arrays[0], perm))
    permutations.append(perm)
    count += 1
    
  signatures = []
  for i in range(len(arrays)):
    signature = []
    for permutation in permutations:
      for j in permutation:
        if(arrays[i][j] == 1):
          signature.append(j)
          break
    signatures.append(signature)
  return signatures
      

In [ ]:
def lsh(table,signature,key_len = 4):
  for i in range(0,len(signature),key_len):
    key = signature[i:i+key_len]
    if(key in table):
      table[key].append(i)
    else:
      table[key] = [i]
  return table
    
  

In [ ]:
def get_32bin_mel(samples):
  return librosa.feature.melspectrogram(y=samples, sr=5512,n_fft=2048, hop_length=64, n_mels=32)

In [ ]:
def get_energy_spec(spec):
  # Transpose 'spec' upfront and get the shifted versions for calculations
  specT = spec.T
  specT_shifted_vertically = np.roll(specT, shift=-1, axis=0)
  specT_shifted_horizontally = np.roll(specT, shift=-1, axis=1)

  # Calculate the differences for current and previous rows
  diff_current = specT - specT_shifted_horizontally
  diff_previous = specT_shifted_vertically - np.roll(specT_shifted_horizontally, shift=-1, axis=0)

  # Compute the energy_spec matrix using NumPy's greater function for element-wise comparison
  energy_spec = np.greater(diff_current - diff_previous, 0).astype(int)
  return energy_spec.T

In [ ]:
dataSet = libs.AudioDataSet.AudioDataset("../data/compressed_index/")




In [ ]:
dataSet.name_list.index('test.mp4')

198

In [ ]:
audio_1 = dataSet[0]


In [ ]:
audio_2 = dataSet[198]

In [ ]:
dataSet.name_list[0]

'023114a7952a10d21c3d5c177b5a8351.mp4'

In [ ]:
audio_3 = dataSet[1]

In [ ]:
audio_2 = audio_1[:,327483:763432]

In [ ]:
len(audio_2[0])

435949

In [ ]:
audio_2 = np.array(audio_2)
sl1 = np.array(audio_3[:,:300000])
sl2 = np.array(audio_3[:,300000:])

res = np.concatenate((sl1,audio_2,sl2),axis=1)

In [ ]:
from moviepy.audio.AudioClip import AudioArrayClip
audio2 = AudioArrayClip(audio_2.T, fps=5512*2)
audio2.write_audiofile('test_2.wav')

MoviePy - Writing audio in test_2.wav


chunk:   0%|          | 0/7544 [00:00<?, ?it/s, now=None]

MoviePy - Done.


In [ ]:
energy_spec1 = get_energy_spec(get_32bin_mel(audio_1[0]))
energy_spec2 = get_energy_spec(get_32bin_mel(audio_2[0]))

In [ ]:
energy_spec1.shape

(32, 140901)

In [ ]:
fingerprint1 = []
for i in range(0,energy_spec1.shape[1],128):
  fingerprint1.append(energy_spec1[:,i:i+128].flatten())
fingerprint2 = []
for i in range(0,energy_spec2.shape[1],128):
  fingerprint2.append(energy_spec2[:,i:i+128].flatten())


In [ ]:
len(fingerprint1)
len(fingerprint2)

1101

In [ ]:
signatures = minHASH(np.concatenate((fingerprint1[:-1],fingerprint2[:-1])), 100)
s1 = signatures[:1100]
s2 = signatures[1100:]

In [ ]:
# m = {}
# for i,p in enumerate(s1):
#   for j,q in enumerate(s2):
#     if len(p)>0 and len(p) == len(q):
#       jac = jaccard_similarity(p,q)
#       if jac > 0.65:
#         minutesI, secondsI = divmod(i*1.48, 60)
#         minutesJ, secondsJ = divmod(j*1.48, 60)
#         formatI = "{:d}:{:02d}".format(int(minutesI), int(secondsI))
#         formatJ = "{:d}:{:02d}".format(int(minutesJ), int(secondsJ))
#         if i in m :
#           m[formatI].append((formatJ,jac))
#         else:
#           m[formatI] = [(formatJ,jac)]
              
    

In [ ]:
lsh_table = {}
for s in s1:
  lsh_table = lsh(lsh_table,s)

lsh_table

NameError: name 's1' is not defined